In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.float_format = '{:.2f}'.format

In [4]:
df_dre_tri_res = pd.read_csv(r"C:\Users\usuario\Desktop\Programacao\financeml\src\data\dados_tratados\dre_tri_res.csv")

In [5]:
df_dre_tri_res

,Unnamed: 0,CNPJ,Ticker,DENOM_CIA,CD_CVM,DT_FIM_EXERC,Resultado Operacional Consolidado,Lucro/Prejuízo Consolidado
0,0,191,BBAS3,BCO BRASIL S.A.,1023,2014-03-31,3717730.00,2774858.00
1,1,191,BBAS3,BCO BRASIL S.A.,1023,2014-06-30,5305500.00,4177761.00
2,2,191,BBAS3,BCO BRASIL S.A.,1023,2014-09-30,6030253.50,5525412.00
3,3,191,BBAS3,BCO BRASIL S.A.,1023,2015-03-31,4595096.00,5675916.00
4,4,191,BBAS3,BCO BRASIL S.A.,1023,2015-06-30,5585760.00,5768503.00
...,...,...,...,...,...,...,...,...
14484,14484,97837181000147,DXCO3,DURATEX S.A.,21091,2015-06-30,124258.50,70214.50
14485,14485,97837181000147,DXCO3,DURATEX S.A.,21091,2015-09-30,159182.50,79926.50
14486,14486,97837181000147,DXCO3,DURATEX S.A.,21091,2016-03-31,-7149.00,-31122.00
14487,14487,97837181000147,DXCO3,DURATEX S.A.,21091,2016-06-30,35062.50,-15678.00


In [6]:
# Converter a coluna de datas para o formato datetime
df_dre_tri_res['DT_FIM_EXERC'] = pd.to_datetime(df_dre_tri_res['DT_FIM_EXERC'])

# Ordenar os dados por empresa (CNPJ_CIA) e data (DT_FIM_EXERC)
df_dre_tri_res = df_dre_tri_res.sort_values(by=['CNPJ', 'DT_FIM_EXERC'])

# Calcular o crescimento trimestre a trimestre do Lucro/Prejuízo Consolidado para cada empresa
df_dre_tri_res['Lucro_Crescimento'] = df_dre_tri_res.groupby('CNPJ')['Lucro/Prejuízo Consolidado'].pct_change() * 100

# Função para calcular o crescimento total do primeiro período até o último para cada empresa
def calculate_total_growth(df):
    # Pegar o primeiro e último valor do 'Lucro/Prejuízo Consolidado' para cada empresa
    first_value = df['Lucro/Prejuízo Consolidado'].iloc[0]
    last_value = df['Lucro/Prejuízo Consolidado'].iloc[-1]
    
    # Calcular o crescimento percentual do primeiro ao último período
    return ((last_value - first_value) / first_value) * 100 if first_value != 0 else None

# Aplicar a função para calcular o crescimento total para cada empresa
total_growth = df_dre_tri_res.groupby('CNPJ').apply(calculate_total_growth).reset_index(name='Crescimento_Total')

# Mesclar os dados de crescimento total de volta ao dataframe original
df_dre_tri_res = df_dre_tri_res.merge(total_growth, on='CNPJ', how='left')

# Definir o período de três anos a partir da última data disponível no dataset
last_date = df_dre_tri_res['DT_FIM_EXERC'].max()
three_years_ago = last_date - pd.DateOffset(years=3)

# Filtrar os dados dos últimos três anos
df_last_three_years = df_dre_tri_res[df_dre_tri_res['DT_FIM_EXERC'] >= three_years_ago]

# Função para calcular o crescimento total nos últimos três anos para cada empresa
def calculate_three_year_growth(df):
    # Pegar o primeiro e último valor do 'Lucro/Prejuízo Consolidado' dentro dos últimos três anos
    first_value = df['Lucro/Prejuízo Consolidado'].iloc[0]
    last_value = df['Lucro/Prejuízo Consolidado'].iloc[-1]
    
    # Calcular o crescimento percentual do primeiro ao último período dos últimos três anos
    return ((last_value - first_value) / first_value) * 100 if first_value != 0 else None

# Aplicar a função para calcular o crescimento nos últimos três anos para cada empresa
three_year_growth = df_last_three_years.groupby('CNPJ').apply(calculate_three_year_growth).reset_index(name='Crescimento_Ultimos_Tres_Anos')

# Mesclar os dados de crescimento dos últimos três anos ao dataframe original
df_dre_tri_res = df_dre_tri_res.merge(three_year_growth, on='CNPJ', how='left')

# Exibir os primeiros dados para verificação
df_dre_tri_res

C:\Users\usuario\AppData\Local\Temp\ipykernel_7588\317543558.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  total_growth = df_dre_tri_res.groupby('CNPJ').apply(calculate_total_growth).reset_index(name='Crescimento_Total')
C:\Users\usuario\AppData\Local\Temp\ipykernel_7588\317543558.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  three_year_growth = df_last_three_years.groupby('CNPJ').apply(calcul

,Unnamed: 0,CNPJ,Ticker,DENOM_CIA,CD_CVM,DT_FIM_EXERC,Resultado Operacional Consolidado,Lucro/Prejuízo Consolidado,Lucro_Crescimento,Crescimento_Total,Crescimento_Ultimos_Tres_Anos
0,0,191,BBAS3,BCO BRASIL S.A.,1023,2014-03-31,3717730.00,2774858.00,NaN,123.85,NaN
1,1,191,BBAS3,BCO BRASIL S.A.,1023,2014-06-30,5305500.00,4177761.00,50.56,123.85,NaN
2,2,191,BBAS3,BCO BRASIL S.A.,1023,2014-09-30,6030253.50,5525412.00,32.26,123.85,NaN
3,3,191,BBAS3,BCO BRASIL S.A.,1023,2015-03-31,4595096.00,5675916.00,2.72,123.85,NaN
4,4,191,BBAS3,BCO BRASIL S.A.,1023,2015-06-30,5585760.00,5768503.00,1.63,123.85,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14484,14475,97837181000147,DXCO3,DEXCO S.A.,21091,2022-06-30,400172.50,280896.00,25.63,178.84,160.70
14485,14476,97837181000147,DXCO3,DEXCO S.A.,21091,2022-09-30,540186.00,350412.50,24.75,178.84,160.70
14486,14477,97837181000147,DXCO3,DEXCO S.A.,21091,2023-03-31,259829.00,147654.00,-57.86,178.84,160.70
14487,14478,97837181000147,DXCO3,DEXCO S.A.,21091,2023-06-30,371032.50,231485.00,56.78,178.84,160.70


In [5]:
df_dre_tri_res.to_csv(r'C:\Users\usuario\Desktop\Programacao\financeml\src\data\dados_com_metricas\df_dreprofit_metricas.csv')